In [2]:
#import packages 

from glob import glob
import pandas as pd
import os
import shutil
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

folder_path="./dataset/subset/"
folders = glob(folder_path+'complete/*')
save_path = 'models/tranfer_best.h5'

train_split=0.7
no_of_images=0

In [14]:
#split data to training and testing

def createFolder(path):
	if os.path.exists(path):
		shutil.rmtree(path)
	os.makedirs(path)

createFolder(folder_path+"train")
createFolder(folder_path+"test")

for folder in folders:
    images = glob(folder+'/*.jpg')
    name = folder.split('/')[-1]
    createFolder(folder_path+"train/"+name)
    createFolder(folder_path+"test/"+name)
    train_images=math.floor(train_split*len(images))
    for i,image in enumerate(images):
        if(i<=train_images-1):
            shutil.copy(image,folder_path+"train/"+name)
        else:
            shutil.copy(image,folder_path+"test/"+name)
        no_of_images+=1
    print(name)


complete\BabyHat
complete\BabyPants
complete\BabyShirt
complete\womencasualshoes
complete\womenlongsleevetop


In [3]:
# Importing the Keras libraries and packages
from keras.models import Sequential,save_model,load_model,Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Input
from keras.layers.core import Dense, Dropout
from keras.applications import ResNet50,nasnet,inception_resnet_v2,Xception
from keras.optimizers import Adam,SGD,RMSprop
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import TensorBoard


# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(139, 139, 3))  # this assumes K.image_data_format() == 'channels_last'
# Initialising the CNN
base_model = inception_resnet_v2.InceptionResNetV2(input_shape = (139, 139, 3), 
                                include_top= False,
                                input_tensor=input_tensor,
                                weights='imagenet')
x = base_model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
predictions = Dense(5,activation='softmax')(x)
classifier = Model(input=base_model.input,output=predictions)

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

classifier.compile(optimizer = opt,
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])
classifier.summary()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 139, 139, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 69, 69, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 69, 69, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 69, 69, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:

# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip=True,
                                  rotation_range = 0.2,
                                  zoom_range = 0.2,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(folder_path+"train/",
                                                 target_size = (139, 139),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(folder_path+"test/",
                                            target_size = (139, 139),
                                            batch_size = 32,
                                            class_mode = 'categorical')

callback=[TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32, write_graph=True, 
                      write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, 
                      embeddings_metadata=None)]

classifier.fit_generator(training_set,steps_per_epoch = 12500/32,
                         epochs = 30,
                         validation_data = test_set,
                         validation_steps = 30,
                         callbacks=callback)

Found 8751 images belonging to 5 classes.
Found 3753 images belonging to 5 classes.
Epoch 1/30


In [7]:
y_pred = classifier.predict_generator(test_set)

In [8]:
y_pred

array([[1.04430341e-03, 6.92757312e-04, 8.24369090e-06, 9.98240113e-01,
        1.45790045e-05],
       [1.49629754e-03, 6.60568901e-07, 9.12943688e-06, 9.98493552e-01,
        3.11282861e-07],
       [6.11243187e-04, 2.15881363e-01, 6.09265387e-01, 1.76994781e-05,
        1.74224257e-01],
       ...,
       [9.98861670e-01, 7.17631599e-04, 2.42137958e-05, 3.17537459e-04,
        7.89396290e-05],
       [1.68993194e-02, 5.80995567e-02, 5.70856512e-01, 2.10768179e-04,
        3.53933841e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00]], dtype=float32)

In [11]:
y_pred_class = np.argmax(y_pred,axis = 1) 
y_pred_class

array([3, 3, 2, ..., 0, 2, 3], dtype=int64)

In [14]:
le = LabelEncoder()
j = []
for i in test_set.filenames: 
    j.append(i.split("\\")[0])
le.fit(j)
valid_actual = le.transform(j)

In [15]:
confusion_mtx = confusion_matrix(valid_actual, y_pred_class) 
print(confusion_mtx)

[[180 131 104 205 175]
 [150  81  67 145 135]
 [132  69  57 173 143]
 [210 125 110 256 230]
 [235 104  95 244 197]]
